In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc




In [4]:
data =  pd.read_csv('bdc_us_mobile_broadband_summary_by_geography_J25_30dec2025.csv')
data.head(5)

,area_data_type,geography_type,geography_id,geography_desc,total_area,mobilebb_3g_area_st_pct,mobilebb_3g_area_iv_pct,mobilebb_4g_area_st_pct,mobilebb_4g_area_iv_pct,mobilebb_5g_spd1_area_st_pct,mobilebb_5g_spd1_area_iv_pct,mobilebb_5g_spd2_area_st_pct,mobilebb_5g_spd2_area_iv_pct
0,Total,National,99,National,9.754824e+06,0.044304,0.030436,0.694594,0.458888,0.498238,0.263329,0.386171,0.179066
1,Total,State,01,Alabama,1.370274e+05,0.000000,0.000000,0.944804,0.634106,0.638194,0.319964,0.472573,0.236973
2,Total,State,02,Alaska,1.846753e+06,0.073956,0.048265,0.082765,0.034033,0.017791,0.006395,0.008757,0.003202
3,Total,State,04,Arizona,2.573078e+05,0.032192,0.021776,0.726136,0.383538,0.409708,0.186351,0.324239,0.126109
4,Total,State,05,Arkansas,1.310934e+05,0.000000,0.000000,0.913444,0.527464,0.662201,0.323019,0.486016,0.231967


In [17]:
data.shape

(20784, 13)

In [47]:
state_data = data[data['geography_type'] == 'State']
state_data = state_data.iloc[:,2:12]
state_data.head(5)

,geography_id,geography_desc,total_area,mobilebb_3g_area_st_pct,mobilebb_3g_area_iv_pct,mobilebb_4g_area_st_pct,mobilebb_4g_area_iv_pct,mobilebb_5g_spd1_area_st_pct,mobilebb_5g_spd1_area_iv_pct,mobilebb_5g_spd2_area_st_pct
1,01,Alabama,1.370274e+05,0.000000,0.000000,0.944804,0.634106,0.638194,0.319964,0.472573
2,02,Alaska,1.846753e+06,0.073956,0.048265,0.082765,0.034033,0.017791,0.006395,0.008757
3,04,Arizona,2.573078e+05,0.032192,0.021776,0.726136,0.383538,0.409708,0.186351,0.324239
4,05,Arkansas,1.310934e+05,0.000000,0.000000,0.913444,0.527464,0.662201,0.323019,0.486016
5,06,California,3.975136e+05,0.000000,0.000000,0.720037,0.474499,0.563454,0.324350,0.472620


In [37]:
county_data = data[data['geography_type']=='County']
county_data = county_data.iloc[:,2:12]
county_data.head(5)

,geography_id,geography_desc,total_area,mobilebb_3g_area_st_pct,mobilebb_3g_area_iv_pct,mobilebb_4g_area_st_pct,mobilebb_4g_area_iv_pct,mobilebb_5g_spd1_area_st_pct,mobilebb_5g_spd1_area_iv_pct,mobilebb_5g_spd2_area_st_pct
57,01001,"Autauga County, AL",1579.987500,0.0,0.0,0.995333,0.654267,0.868867,0.347867,0.638800
58,01003,"Baldwin County, AL",5165.716465,0.0,0.0,0.933669,0.700237,0.756352,0.516476,0.601260
59,01005,"Barbour County, AL",2407.058290,0.0,0.0,0.967180,0.562533,0.480308,0.139725,0.329730
60,01007,"Bibb County, AL",1630.231102,0.0,0.0,0.966014,0.526006,0.474317,0.123538,0.286683
61,01009,"Blount County, AL",1720.185058,0.0,0.0,0.999143,0.876186,0.945257,0.630274,0.694201


In [38]:
df_states = pd.DataFrame(state_data)
df_counties = pd.DataFrame(county_data)

In [39]:
df_states['4G_pct'] = df_states['mobilebb_4g_area_st_pct'] * 100
df_states['5G_Speed1_pct'] = df_states['mobilebb_5g_spd1_area_st_pct'] * 100
df_states['5G_Speed2_pct'] = df_states['mobilebb_5g_spd2_area_st_pct'] * 100

df_counties['4G_pct'] = df_counties['mobilebb_4g_area_st_pct'] * 100
df_counties['5G_Speed1_pct'] = df_counties['mobilebb_5g_spd1_area_st_pct'] * 100
df_counties['5G_Speed2_pct'] = df_counties['mobilebb_5g_spd2_area_st_pct'] * 100

# Calculate additional metrics
df_states['5G_gap'] = df_states['4G_pct'] - df_states['5G_Speed1_pct']

In [48]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def create_state_choropleth():
    """Create US state choropleth map"""
    fig = go.Figure(data=go.Choropleth(
        locations=df_states['geography_desc'],
        z=df_states['4G_pct'],
        locationmode='USA-states',
        colorscale='RdYlGn',
        colorbar_title="4G Coverage %",
        text=df_states['geography_desc'],
        hovertemplate='<b>%{text}</b><br>4G Coverage: %{z:.1f}%<extra></extra>'
    ))
    
    fig.update_layout(
        title_text='US States 4G Coverage Map',
        geo_scope='usa',
        height=500,
        margin=dict(l=0, r=0, t=40, b=0)
    )
    return fig
def create_top_performers():
    """Create top performers bar chart"""
    top5 = df_states.nlargest(5, '4G_pct').sort_values('4G_pct')
    
    fig = go.Figure(go.Bar(
        y=top5['geography_desc'],
        x=top5['4G_pct'],
        orientation='h',
        marker=dict(
            color=top5['4G_pct'],
            colorscale='RdYlGn',
            showscale=False
        ),
        text=top5['4G_pct'].apply(lambda x: f'{x:.1f}%'),
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Top 5 States - 4G Coverage',
        xaxis_title='Coverage (%)',
        yaxis_title='',
        height=400,
        margin=dict(l=150, r=50, t=40, b=40)
    )
    return fig

def create_technology_comparison():
    """Create technology comparison chart"""
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name='4G',
        x=df_states['geography_desc'],
        y=df_states['4G_pct'],
        marker_color='#3498db'
    ))
    
    fig.add_trace(go.Bar(
        name='5G Speed 1',
        x=df_states['geography_desc'],
        y=df_states['5G_Speed1_pct'],
        marker_color='#9b59b6'
    ))
    
    fig.add_trace(go.Bar(
        name='5G Speed 2',
        x=df_states['geography_desc'],
        y=df_states['5G_Speed2_pct'],
        marker_color='#e74c3c'
    ))
    
    fig.update_layout(
        title='Technology Coverage Comparison by State',
        xaxis_title='State',
        yaxis_title='Coverage (%)',
        barmode='group',
        height=500,
        xaxis_tickangle=-45,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

In [49]:
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Broadband Coverage Analysis Dashboard", className="text-center mb-4 mt-4"),
            html.P("Interactive analysis of mobile broadband infrastructure across the United States", 
                   className="text-center text-muted mb-4")
        ])
    ]),
    
    # Key Metrics Row
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{df_states['4G_pct'].mean():.1f}%", className="card-title text-primary"),
                    html.P("Avg 4G Coverage (States)", className="card-text")
                ])
            ], className="mb-3")
        ], width=3),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{df_states['5G_Speed1_pct'].mean():.1f}%", className="card-title text-success"),
                    html.P("Avg 5G Speed 1 (States)", className="card-text")
                ])
            ], className="mb-3")
        ], width=3),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{df_counties['4G_pct'].mean():.1f}%", className="card-title text-info"),
                    html.P("Avg 4G Coverage (Counties)", className="card-text")
                ])
            ], className="mb-3")
        ], width=3),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{len(df_states)}", className="card-title text-warning"),
                    html.P("States Analyzed", className="card-text")
                ])
            ], className="mb-3")
        ], width=3),
    ]),
    
    # Tabs
    dbc.Tabs([
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=create_state_choropleth())
                ], width=12)
            ], className="mt-3"),
            
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=create_top_performers())
                ], width=6),
                
                dbc.Col([
                    dcc.Graph(figure=create_county_top_performers())
                ], width=6)
            ], className="mt-3"),
            
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=create_technology_comparison())
                ], width=12)
            ], className="mt-3")
        ], label="Overview", tab_id="overview"),
        
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=create_area_vs_coverage())
                ], width=6),
                
                dbc.Col([
                    dcc.Graph(figure=create_5g_comparison())
                ], width=6)
            ], className="mt-3"),
            
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=create_correlation_heatmap())
                ], width=6),
                
                dbc.Col([
                    dcc.Graph(figure=create_distribution_histogram())
                ], width=6)
            ], className="mt-3")
        ], label="Analysis", tab_id="analysis"),
        
        dbc.Tab([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=create_coverage_gap())
                ], width=12)
            ], className="mt-3"),
            
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Key Findings", className="bg-primary text-white"),
                        dbc.CardBody([
                            html.Ul([
                                html.Li(f"Best State: {df_states.loc[df_states['4G_pct'].idxmax(), 'geography_desc']} ({df_states['4G_pct'].max():.1f}%)"),
                                html.Li(f"Worst State: {df_states.loc[df_states['4G_pct'].idxmin(), 'geography_desc']} ({df_states['4G_pct'].min():.1f}%)"),
                                html.Li(f"Average 4G to 5G Gap: {df_states['5G_gap'].mean():.1f}%"),
                                html.Li(f"States with >90% 4G: {len(df_states[df_states['4G_pct'] >= 90])} of {len(df_states)}"),
                                html.Li(f"Correlation (Area vs 4G): {df_states['total_area'].corr(df_states['4G_pct']):.3f}"),
                                html.Li(f"Best County: {df_counties.loc[df_counties['4G_pct'].idxmax(), 'geography_desc']} ({df_counties['4G_pct'].max():.1f}%)"),
                            ])
                        ])
                    ])
                ], width=6),
                
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Statistical Summary", className="bg-success text-white"),
                        dbc.CardBody([
                            html.H6("State-Level Statistics", className="card-subtitle mb-2"),
                            html.P(f"4G - Mean: {df_states['4G_pct'].mean():.1f}%, Median: {df_states['4G_pct'].median():.1f}%, Std: {df_states['4G_pct'].std():.1f}%"),
                            html.P(f"5G Speed 1 - Mean: {df_states['5G_Speed1_pct'].mean():.1f}%, Std: {df_states['5G_Speed1_pct'].std():.1f}%"),
                            html.Hr(),
                            html.H6("County-Level Statistics", className="card-subtitle mb-2"),
                            html.P(f"4G - Mean: {df_counties['4G_pct'].mean():.1f}%, Median: {df_counties['4G_pct'].median():.1f}%, Std: {df_counties['4G_pct'].std():.1f}%"),
                            html.P(f"5G Speed 1 - Mean: {df_counties['5G_Speed1_pct'].mean():.1f}%, Std: {df_counties['5G_Speed1_pct'].std():.1f}%"),
                        ])
                    ])
                ], width=6)
            ], className="mt-3")
        ], label="Insights", tab_id="insights")
    ], id="tabs", active_tab="overview")
], fluid=True)

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

NameError: name 'create_county_top_performers' is not defined